# 🔍 Prediction using Random Forest

This notebook performs supervised regression using a Random Forest model to predict **** from environmental and categorical variables.

---

## 🧪 Objectives

- Predict SOC based on environmental predictors and categorical treatment factors.
- Rank feature importance.
- Visualize and interpret model performance, residuals, and SHAP values.

---

## 📦 Workflow

1. **Data Preparation**  
   - Load and clean CSV data.  
   - One-hot encode categorical columns: `Year`, `Species`, `Tre`, `Sub`.

2. **Model Training**  
   - Random Forest Regression (100 trees, with OOB scoring).  
   - Train/test split (70/30).

3. **Model Evaluation**  
   - Metrics: MSE, RMSE, MAE, R².  
   - Feature importance analysis.  
   - Residual and complexity analysis.  
   - SHAP and Partial Dependence Plots.

4. **Feature Selection**  
   - Select features with importance > 0.01.  
   - Re-train and cross-validate with selected features.

---

## 📊 Outputs

- `feature_importance04.csv`: Importance scores of predictors.
- `feature_importance_plot04.png`: Horizontal bar chart.
- `residual_plot04.png`: Actual vs. predicted plot.
- `model_complexity_plot04.png`: MSE vs. number of trees.
- `pdp_plot04.png`: Partial dependence plots.
- SHAP summary plot for feature impact visualization.

---

## 💡 Notes

- SHAP is used for model interpretability.
- Model complexity curve helps avoid overfitting.
- Suitable for environmental modeling and field experiment datasets.

---

## 🔧 Requirements

- `pandas`, `scikit-learn`, `matplotlib`, `shap`, `numpy`

---

## 📁 Example Directory Structure



In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.inspection import PartialDependenceDisplay
import shap
import matplotlib.pyplot as plt
import numpy as np

# Load the CSV file
file_path = r'..SoC.csv'  # Replace with your own path
df = pd.read_csv(file_path)

# Remove whitespace from column names
df.columns = df.columns.str.strip()

# Ensure the target column 'SOC' exists
if 'SOC' not in df.columns:
    raise KeyError("SOC column not found in the dataset.")

# One-hot encode categorical variables as interaction terms
categorical_cols = ['Year', 'Species', 'Tre', 'Sub']
X_categorical = pd.get_dummies(df[categorical_cols], drop_first=True)

# Separate numeric features and response variable
X_numeric = df.drop(columns=['SOC', 'Year', 'Species', 'Tre', 'Sub'])
X = pd.concat([X_numeric, X_categorical], axis=1)
Y = df['SOC']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Initialize and train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, oob_score=True)
rf_model.fit(X_train, y_train)

# Predict and evaluate model performance
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R²): {r2}')
print(f'OOB Score (R²): {rf_model.oob_score_}')

# Analyze feature importance
feature_importances = rf_model.feature_importances_
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Filter features with importance above threshold
threshold = 0.01
selected_features = importance_df[importance_df['Importance'] > threshold]['Feature'].tolist()
X_selected = X[selected_features]

print(f'Selected Features: {selected_features}')

# Perform K-fold cross-validation on selected features
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf_model, X_selected, Y, cv=kf, scoring='r2')
print(f'Cross-validated R²: {cv_scores.mean()} ± {cv_scores.std()}')

# SHAP analysis
X_train_selected, X_test_selected, y_train, y_test = train_test_split(X_selected, Y, test_size=0.3, random_state=42)
rf_model.fit(X_train_selected, y_train)
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test_selected)
shap.summary_plot(shap_values, X_test_selected)

# Save feature importance to CSV
importance_csv_path = r'..feature_importance04.csv'
importance_df.to_csv(importance_csv_path, index=False)
print(f'Feature importance saved to {importance_csv_path}')

# Plot feature importance
plt.figure(figsize=(12, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Feature Importance')
plt.title(f'Feature Importance from Random Forest (R² = {round(r2, 2)})')
plt.gca().invert_yaxis()

# Annotate bar values
for index, value in enumerate(importance_df['Importance']):
    plt.text(value, index, f'{value:.4f}', va='center')

plt.tight_layout()
plt.savefig(r'..feature_importance_plot04.png')
plt.show()

# Residual analysis
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red')
plt.xlabel('Actual SOC')
plt.ylabel('Predicted SOC')
plt.title('Actual vs Predicted SOC')
plt.tight_layout()
plt.savefig(r'..residual_plot04.png')
plt.show()

# Model complexity analysis: test error vs. number of trees
errors = []
for n in range(10, 200, 10):
    rf = RandomForestRegressor(n_estimators=n, random_state=42)
    rf.fit(X_train, y_train)
    y_pred_test = rf.predict(X_test)
    errors.append(mean_squared_error(y_test, y_pred_test))

plt.figure(figsize=(10, 6))
plt.plot(range(10, 200, 10), errors, marker='o')
plt.xlabel('Number of Trees')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Model Complexity Analysis')
plt.tight_layout()
plt.savefig(r'..model_complexity_plot04.png')
plt.show()

# Partial Dependence Plot (PDP)
fig, ax = plt.subplots(figsize=(14, 10))
PartialDependenceDisplay.from_estimator(
    rf_model, X_train_selected, features=X_selected.columns, feature_names=X_selected.columns, ax=ax
)
plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.tight_layout()
plt.savefig(r'..pdp_plot04.png')
plt.show()
